In [1]:
from tkinter import *
import csv
import tweepy
from textblob import TextBlob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer


In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marcos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
root=Tk()    # it provides the basic window for display
root.title("Movie Success Prediction")

''

In [4]:
def cleanAna():
    polar=[]
    subject=[]
    try:
        with open("Dataset_Major.csv","a") as csvfile:
            writer=csv.writer(csvfile)
            #writer.writerow(["Movie Name","Genre","Actor Name","Polarity","Subjectivity"])
            for tweet in tweepy.Cursor(api.search,q=name.get(),lang="en").items(1000):
                review=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.text)
                review=review.lower()
                review=review.split()
                ps=PorterStemmer()
                review=[ps.stem(word) for word in review if not word in set(stopwords.words("english"))]
                review=" ".join(review)
                analysis=TextBlob(review)
                polar.append(analysis.sentiment.polarity)
                subject.append(analysis.sentiment.subjectivity)
            
            polarScore=float(sum(polar)/len(polar))
            polar.clear()
            subjectScore=float(sum(subject)/len(subject))
            subject.clear()
            print(polarScore)
            writer.writerow([name.get(),genre.get(ACTIVE),actorName1.get(),polarScore,subjectScore,actorName2.get(),1])
            csv.close()
            
    except Exception:
        return

In [5]:
def learnSVR():
    try:
        dataset = pd.read_csv('Dataset_Major.csv')
        #dataset1= pd.read_csv("Dataset.csv")
        X = dataset.iloc[:,[3,4]].values
        y = dataset.iloc[:, 6].values
    
        #X_test=dataset1.iloc[:,[3,4]].values

        # Splitting the dataset into the Training set and Test set
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

        # Feature Scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        sc_y = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.fit_transform(X_test)
        y_train = sc_y.fit_transform(y_train.reshape(len(y_train),1))

        # Fitting SVR to the dataset
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)

        # Predicting a new result
        y_pred = sc_y.inverse_transform(regressor.predict(X_test))
        
        return y_pred[-1]
    except Exception as e:
        print("Exception occured in SVR",e)

In [6]:
def learnDec():
    try:
        dataset = pd.read_csv('Dataset_Major.csv')
        #dataset1= pd.read_csv("Dataset.csv")
        X = dataset.iloc[:,[3,4]].values
        y = dataset.iloc[:, 6].values
    
        #X_test=dataset1.iloc[:,[3,4]].values
    
    
        # Splitting the dataset into the Training set and Test set
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    
        # Feature Scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
    
        # Fitting Decision Tree Regression to the dataset
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
    
        y_pred = regressor.predict(X_test)
        return y_pred[-1]
    except Exception as e:
        print("Exception occured in Decision",e)
    

In [7]:
def learnRandom():
    try:
        dataset = pd.read_csv('Dataset_Major.csv')
        #dataset1= pd.read_csv("Dataset.csv")
        X = dataset.iloc[:,[3,4]].values
        y = dataset.iloc[:, 6].values
    
        #X_test=dataset1.iloc[:,[3,4]].values

        # Splitting the dataset into the Training set and Test set
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

        # Feature Scaling
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        #sc_y = StandardScaler()
        #y_train = sc_y.fit_transform(y_train.reshape(-1,1))

        # Fitting Random Forest Regression to the dataset
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)

        # Predicting a new result
        y_pred = regressor.predict(X_test)
        return y_pred[-1]
    except Exception as e:
        print("Exception occured in Random",e)

In [8]:

def tweets():
    try:
        cleanAna()
        a=learnSVR()
        b=learnDec()
        c=learnRandom()
        label1.config(text="Using SVR: "+str(a))
        label2.config(text="Using Decision Tree Regression: "+str(b))
        label3.config(text="Using Random Forest Regression: "+str(c))
        if a>=6 or b>=6 or c>=6:
            label4.config(text="Success")
        elif a in range(4,6) or b in range(4,6) or c in range(4,6):
            label4.config(text="Average")
        else:
            label4.config(text="Flop")
    except Exception as e:
        print("Exception occured in tweets",e)
    

In [9]:
apiKey="0PaDbNyGOmxziJFzFL427dcoF"
apiSecretKey="YxzTOsvaEnyhrwDwqswLzsqS36BuEPML0d2fL8yGS4FqU8OjNw"
accessToken="4904328277-XRU4LIs01e7qiyvKTs4gGBGExyJrkT8xp4KckFy"
accessTokenSecret="GuDJ9IjGzWvsKFYMSQUAoXNCclXCghUAiuDdM73NIEukr"

auth = tweepy.OAuthHandler(apiKey,apiSecretKey)
auth.set_access_token(accessToken,accessTokenSecret)
api=tweepy.API(auth)


In [10]:
root.geometry("400x900")

labelName=Label(root,text="Enter the name of the movie",padx=15,pady=10)
name=Entry(root,width=50,borderwidth=5)


labelActor1=Label(root,text="Enter the name of 1st actor",padx=15,pady=10)
actorName1=Entry(root,width=50,borderwidth=5)
labelActor2=Label(root,text="Enter the name of 2nd actor",padx=15,pady=10)
actorName2=Entry(root,width=50,borderwidth=5)

labelGenre=Label(root,text="Select the genre of the movie",padx=15,pady=10)
genre=Listbox(root,height=5,bd=5,selectmode="Single")
genre.insert(1,"Action")
genre.insert(2,"Drama")
genre.insert(3,"Horror")
genre.insert(4,"Comedy")
genre.insert(5,"Romantic")

labelDate=Label(root,text="Enter the year of release",padx=15,pady=10)
date=Entry(root,width=50,borderwidth=5)

labelPred=Label(root,text="Imdb Score predicted",padx=15,pady=10)
label1=Label(root,padx=15,pady=10)
label2=Label(root,padx=15,pady=10)
label3=Label(root,padx=15,pady=10)
label4=Label(root,padx=15,pady=10)


In [11]:
# .pack() or .grid()
labelName.pack()
name.pack()

labelActor1.pack()
actorName1.pack()
labelActor2.pack()
actorName2.pack()

labelGenre.pack()
genre.pack()

labelDate.pack()
date.pack()
labelPred.pack()
label1.pack()
label2.pack()
label3.pack()
label4.pack()


In [12]:
button_predict=Button(root,text="Predict",bg="#7EC0EE",padx=15,pady=10,command=tweets) 
button_predict.pack()

button_quit=Button(root,text="Exit",bg="#7EC0EE",padx=15,pady=10,command=root.destroy)
button_quit.pack()

In [ ]:
root.mainloop()  #it is the infinite loop to view the window

C:\Users\Marcos\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
